In [2]:
library(rtracklayer)

In [4]:
#FOR WHEN STARTING WITH SOME FILTERED SNPS FROM NOSE-JAW scores

In [4]:
snp_df <- read.csv('white_SNPs_filtered_nose_jaw_scores.csv')

In [5]:
# Suppose your SNP dataframe is called snp_df, and column CHR_POS_RS looks like "1_1234567_rs1234"
snp_df$chr <- sub("^(\\d+)_.*", "\\1", snp_df$CHR_POS_RS)
snp_df$pos <- as.integer(sub("^\\d+_(\\d+)_.*", "\\1", snp_df$CHR_POS_RS))
snp_df$rsid <- sub("^\\d+_\\d+_(.*)$", "\\1", snp_df$CHR_POS_RS)

# Ensure chromosome names match UCSC format
snp_df$chr <- paste0("chr", snp_df$chr)

# Make GRanges (SNPs are 1bp intervals)
snp_gr <- GRanges(
  seqnames = snp_df$chr,
  ranges = IRanges(start = snp_df$pos, end = snp_df$pos),
  rsid = snp_df$rsid
)


In [12]:
setwd('/home/alek/FACE/DATA/GWAS/white_summary_stats')

In [2]:
# Load the chain file if needed
#chain <- import.chain("hg19ToHg38.over.chain")

In [16]:

# Liftover
snp_gr_hg38 <- liftOver(snp_gr, chain)

# Keep only mappings with exactly 1 result
snp_gr_hg38 <- unlist(snp_gr_hg38[lengths(snp_gr_hg38) == 1])

Discarding unchained sequences: chr23



In [17]:
snp_hg38_df <- data.frame(
  chr = as.character(seqnames(snp_gr_hg38)),
  pos = start(snp_gr_hg38),
  rsid = snp_gr_hg38$rsid
)


In [20]:
# Merge hg38 coordinates into your original snp_df by 'rsid'
snp_df_hg38 <- merge(
  snp_df,
  snp_hg38_df,
  by = "rsid",
  all.x = TRUE,   # keep all SNPs from your original file
  suffixes = c("_hg19", "_hg38")
)



In [27]:
snp_df_hg38$chr_hg38 <- sub("^chr", "", snp_df_hg38$chr_hg38)

# Optionally, create a new CHR_POS_RS column for hg38
snp_df_hg38$CHR_POS_RS_hg38 <- paste(snp_df_hg38$chr_hg38, snp_df_hg38$pos_hg38, snp_df_hg38$rsid, sep = "_")


In [30]:
snp_df_hg38$CHR_POS_RS <- snp_df_hg38$CHR_POS_RS_hg38
snp_df_hg38$CHR_POS_RS_hg38 <- NULL


In [31]:
write.csv(snp_df_hg38, 'white_SNPs_filtered_nose_jaw_scores_hg38.csv')

In [ ]:
#ANOTHER FILE NOW , FOR WHEN STARTING WITH RAW WHITE SNPs
snp_df1 <- read.csv('pvalues_MinP_filtered.csv')
snp_df2 <- read.csv('snp_info_filtered.csv')

In [36]:
snp_df <- cbind(snp_df2, snp_df1)

In [1]:

# Ensure chromosome names match UCSC format
#snp_df$CHR <- paste0("chr", snp_df$CHR)

#DEPENDS ON THE FORMAT OF THE INPUT


In [43]:
# Make GRanges (SNPs are 1bp intervals)
snp_gr <- GRanges(
  seqnames = snp_df$CHR,
  ranges = IRanges(start = snp_df$POS, end = snp_df$POS),
  rsid = snp_df$RS
)

setwd('/home/alek/FACE/DATA/GWAS/white_summary_stats')

chain <- import.chain("hg19ToHg38.over.chain")

snp_gr_hg38 <- liftOver(snp_gr, chain)
snp_gr_hg38 <- unlist(snp_gr_hg38[lengths(snp_gr_hg38) == 1])


snp_hg38_df <- data.frame(
  chr = as.character(seqnames(snp_gr_hg38)),
  pos = start(snp_gr_hg38),
  rsid = snp_gr_hg38$rsid
)

if(!"rsid" %in% colnames(snp_df)) colnames(snp_df)[colnames(snp_df) == "RS"] <- "rsid"

snp_df_hg38 <- merge(
  snp_df,
  snp_hg38_df,
  by = "rsid",
  all.x = TRUE,   # keep all SNPs from your original file
  suffixes = c("_hg19", "_hg38")
)

Discarding unchained sequences: chr23



In [45]:
colnames(snp_df_hg38)[colnames(snp_df_hg38) == "POS"] <- "POS_hg19"
colnames(snp_df_hg38)[colnames(snp_df_hg38) == "pos"] <- "pos_hg38"
colnames(snp_df_hg38)[colnames(snp_df_hg38) == "chr"] <- "chr_hg38"
colnames(snp_df_hg38)[colnames(snp_df_hg38) == "CHR"] <- "CHR_hg19"

In [47]:
write.csv(snp_df_hg38, 'WHITE_hg38.csv')